## 1) Libraries Installation
##### The cell below is to help you keep track the libraries used and install them quickly.
##### Ensure the correct library names are used, and follow the syntax: **%pip install PACKAGE_NAME**.

In [1]:
#install.packages('tsibble')
#install.packages('fable')
#install.packages('fabletools')
#install.packages('BVAR')
#install.packages('feasts')
#install.packages("urca")
library(tidyverse)
library(tsibble)
library(fable)
library(fabletools)
library(stats)
library(BVAR)
library(feasts)
library(urca)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Registered S3 method overwritten by 'tsibble':
  method               from 
  as_tibble.grouped_df dplyr


Attaching package: ‘tsibble’


The following object is masked from ‘package:lubridate’:

    interval


The following objects are masked from ‘package:base’:

    intersect, setdiff, union


Loading required package: fabletools



## 2) Main Section for Code
### Preparing dataset for analysis and applying the relevant stationarity-inducing transformations to the variables.

In [10]:
data = read.csv("https://raw.githubusercontent.com/dsesc-acads/Databusters/refs/heads/main/Quarterly%20Data.csv")
data = data[-c(1:38),]
data <- data %>%
  dplyr::select('sasdate', "GDPC1", 'GS10TB3Mx', 'CLAIMSx', 'ANDENOx', 'UMCSENTx') %>%
  mutate(sasdate = yearquarter(sasdate))
transformed <- fred_transform(data[,-1], type=c('fred_qd'), codes=c(5,1,5,5,2))
transformed <- cbind(data$sasdate[-1], transformed) %>%
  rename(date = 'data$sasdate[-1]') %>%
  mutate(GDPC1 = 4 * GDPC1) %>%
  as_tsibble(index=date)

transformed <- transformed %>%
  mutate(across(c(GS10TB3Mx, CLAIMSx, ANDENOx, UMCSENTx, GDPC1),
                 list(lag1 = ~ lag(., 1), lag2 = ~ lag(., 2),lag3 = ~ lag(., 3), lag4 = ~ lag(., 4)),
                 .names = "{col}_{fn}"))

transformed <- transformed[-c(1:4),]

## 3) AR Lag Selection
### Preparing dataset for analysis and applying the relevant stationarity-inducing transformations to the variables.

In [11]:
ar_lags <- transformed[1:215,] %>%
  model(ar1 = AR(GDPC1 ~ order(1)),
        ar2 = AR(GDPC1 ~ order(2)),
        ar3 = AR(GDPC1 ~ order(3)),
        ar4 = AR(GDPC1 ~ order(4)),
        ar5 = AR(GDPC1 ~ order(5)),
        ar6 = AR(GDPC1 ~ order(6)),
        ar7 = AR(GDPC1 ~ order(7)),
        ar8 = AR(GDPC1 ~ order(8)))

glance(ar_lags) %>%
  arrange(AIC)

ar_lags %>%
  fabletools::forecast(new_data=transformed[216:219,]) %>%
  accuracy(transformed)

.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
ar1,19.39081,3.887315,3.943918,10.62859,213
ar2,19.41131,6.114516,6.228260,16.22643,212
ar3,19.38513,7.824285,8.014761,21.30684,211
ar4,19.42438,10.259214,10.546295,27.11240,210
ar5,19.49947,13.088720,13.492567,33.31255,209
ar8,19.07915,14.403660,15.281709,44.73940,206
ar6,19.51098,15.215632,15.756695,38.81010,208
ar7,19.34867,15.419565,16.118594,42.38467,207


.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ar1,Test,0.4649493,0.8348992,0.5954848,10.95524,16.34803,0.1590267,0.1343137,-0.2781114
ar2,Test,0.4542497,0.8372748,0.6037337,10.52520,16.70080,0.1612296,0.1346959,-0.2835602
ar3,Test,0.4484738,0.8205455,0.6083450,10.43221,17.03695,0.1624611,0.1320046,-0.3238528
ar4,Test,0.4454774,0.8117210,0.6028504,10.39569,16.89720,0.1609937,0.1305850,-0.3419946
ar5,Test,0.4365601,0.8072075,0.5950677,10.09854,16.64694,0.1589153,0.1298589,-0.3364216
ar6,Test,0.4408962,0.6741048,0.4544737,11.46929,12.03021,0.1213691,0.1084461,-0.4090615
ar7,Test,0.4472806,0.7368671,0.4608549,11.43507,11.99586,0.1230732,0.1185429,-0.5282952
ar8,Test,0.6605542,0.8423000,0.6605542,18.82127,18.82127,0.1764038,0.1355043,-0.4503462


Selection of lags of other predictors:

In [12]:
### Term Spread
ardl_spread<- transformed[1:215,] %>%
  model(AR(GDPC1 ~ order(1)),
        AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1),
        AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2),
        AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + GS10TB3Mx_lag3),
        AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + GS10TB3Mx_lag3 + GS10TB3Mx_lag4))

glance(ardl_spread) %>%
  arrange(AIC)

.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2),18.18772,-5.884038,-5.693562,7.598514,211
AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + GS10TB3Mx_lag3),18.15880,-4.226142,-3.939061,12.627048,210
AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + GS10TB3Mx_lag3 + GS10TB3Mx_lag4),18.12433,-2.634605,-2.230759,17.589223,209
AR(GDPC1 ~ order(1) + GS10TB3Mx_lag1),18.69201,-2.003878,-1.890134,8.108036,212
AR(GDPC1 ~ order(1)),19.39081,3.887315,3.943918,10.628591,213


In [13]:
### Unemployment claims
ardl_claims<- transformed[1:215,] %>%
  model(AR(GDPC1 ~ order(1)),
        AR(GDPC1 ~ order(1) + CLAIMSx_lag1),
        AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2),
        AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3),
        AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4))

glance(ardl_claims) %>%
  arrange(AIC)


.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4),15.04411,-42.682365,-42.278518,-22.45854,209
AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3),15.26594,-41.535387,-41.248306,-24.68220,210
AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2),15.45349,-40.910092,-40.719616,-27.42754,211
AR(GDPC1 ~ order(1) + CLAIMSx_lag1),16.25040,-32.099198,-31.985454,-21.98728,212
AR(GDPC1 ~ order(1)),19.39081,3.887315,3.943918,10.62859,213


In [14]:
### Housing starts
ardl_hstarts<- transformed[1:215,] %>%
  model(AR(GDPC1 ~ order(1)),
        AR(GDPC1 ~ order(1) + ANDENOx_lag1),
        AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2),
        AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2 + ANDENOx_lag3),
        AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2 + ANDENOx_lag3 + ANDENOx_lag4))

glance(ardl_hstarts) %>%
  arrange(AIC)


.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2),18.53431,-1.8254707,-1.6349945,11.65708,211
AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2 + ANDENOx_lag3),18.47990,-0.4575456,-0.1704643,16.39564,210
AR(GDPC1 ~ order(1) + ANDENOx_lag1 + ANDENOx_lag2 + ANDENOx_lag3 + ANDENOx_lag4),18.43130,0.9763187,1.3801648,21.20015,209
AR(GDPC1 ~ order(1)),19.39081,3.8873146,3.9439184,10.62859,213
AR(GDPC1 ~ order(1) + ANDENOx_lag1),19.39060,5.8850083,5.9987524,15.99692,212


In [15]:
### Consumer Sentiment Index
ardl_csent <- transformed[1:215,] %>%
  model(AR(GDPC1 ~ order(1)),
        AR(GDPC1 ~ order(1) + UMCSENTx_lag1),
        AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2),
        AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3),
        AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4))

glance(ardl_csent) %>%
  arrange(AIC)


.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4),18.51930,2.000300,2.404146,22.22413,209
AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3),18.81839,3.444946,3.732027,20.29814,210
AR(GDPC1 ~ order(1)),19.39081,3.887315,3.943918,10.62859,213
AR(GDPC1 ~ order(1) + UMCSENTx_lag1 + UMCSENTx_lag2),19.06065,4.195016,4.385493,17.67757,211
AR(GDPC1 ~ order(1) + UMCSENTx_lag1),19.35868,5.530801,5.644545,15.64272,212


In [16]:
ardl_combined <- transformed[1:215,] %>%
  model(AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4 + ANDENOx_lag1 + ANDENOx_lag2 + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4))

glance(ardl_combined)

ardl_combined %>%
  fabletools::forecast(new_data=transformed[216:219,]) %>%
  accuracy(transformed)



.model,sigma2,AIC,AICc,BIC,dof
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4 + ANDENOx_lag1 + ANDENOx_lag2 + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4),11.68745,-80.96372,-78.86372,-33.77479,201


.model,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
AR(GDPC1 ~ order(1) + CLAIMSx_lag1 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4 + ANDENOx_lag1 + ANDENOx_lag2 + GS10TB3Mx_lag1 + GS10TB3Mx_lag2 + UMCSENTx_lag1 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4),Test,1.00991,1.569894,1.420098,30.86572,43.9194,0.3792432,0.2525554,-0.5208495


In [18]:
var <- transformed[1:215,] %>%
  model(var1 = VAR(vars(GDPC1, CLAIMSx, UMCSENTx, GS10TB3Mx, ANDENOx) ~ AR(1)),
        var2 = VAR(vars(GDPC1, CLAIMSx, UMCSENTx, GS10TB3Mx, ANDENOx) ~ AR(2)),
        var3 = VAR(vars(GDPC1, CLAIMSx, UMCSENTx, GS10TB3Mx, ANDENOx) ~ AR(3)),
        var4 = VAR(vars(GDPC1, CLAIMSx, UMCSENTx, GS10TB3Mx, ANDENOx) ~ AR(4)))


for (i in 1:4){
   print(sprintf("AIC of VAR %s is %s", i, var[[i]][[1]][["fit"]][["fit"]][["AIC"]]))
}

var %>%
  fabletools::forecast(new_data=transformed[216:219,]) %>%
  accuracy(transformed) %>%
  filter(.response == 'GDPC1') %>%
  arrange(RMSE)


[1] "AIC of VAR 1 is 5928.44235391567"
[1] "AIC of VAR 2 is 5847.34857094617"
[1] "AIC of VAR 3 is 5853.57678037953"
[1] "AIC of VAR 4 is 5826.9000853482"


.model,.response,.type,ME,RMSE,MAE,MPE,MAPE,MASE,RMSSE,ACF1
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
var1,GDPC1,Test,1.548758,1.674906,1.548758,47.56132,47.56132,0.4136022,0.2694491,-0.30773978
var2,GDPC1,Test,1.642458,1.852914,1.642458,49.96256,49.96256,0.4386253,0.2980860,-0.03717379
var3,GDPC1,Test,2.127077,2.242014,2.127077,67.70228,67.70228,0.5680449,0.3606822,-0.03163843
var4,GDPC1,Test,2.699485,2.794232,2.699485,85.00987,85.00987,0.7209088,0.4495199,-0.43835207


## Forecasting

In [50]:
## Q1 2024
### AR
ar1 <-transformed %>%
  model(ar1 = AR(GDPC1 ~ order(1)))

ar1_fc <- ar1 %>%
  fabletools::forecast(h=5)
ar1_fc


### ARDL using direct h-step forecasting
transformed_ardl <- transformed %>%
  mutate(across(c(GDPC1, GS10TB3Mx, CLAIMSx, ANDENOx, UMCSENTx),
                list(lag5 = ~ lag(., 5), lag6 = ~ lag(., 6),lag7 = ~ lag(., 7), lag8 = ~ lag(., 8), lag9 = ~ lag(., 9)),
                .names = "{col}_{fn}"))

last_obs <- transformed_ardl %>% slice_tail(n = 3)
future_data1 <- new_data(transformed, n = 1, keep_all=TRUE) %>%
  mutate(
    date = yearquarter("2025 Q1"),
    GDPC1_lag2 = last_obs$GDPC1[3],
    CLAIMSx_lag2 = last_obs$CLAIMSx_lag1[3],
    CLAIMSx_lag3 = last_obs$CLAIMSx_lag2[3],  # Correctly getting the t-2 value
    CLAIMSx_lag4 = last_obs$CLAIMSx_lag3[3],
    CLAIMSx_lag5 = last_obs$CLAIMSx_lag4[3],
    ANDENOx_lag2 = last_obs$ANDENOx_lag1[3],
    ANDENOx_lag3 = last_obs$ANDENOx_lag2[3],
    GS10TB3Mx_lag2 = last_obs$GS10TB3Mx_lag1[3],
    GS10TB3Mx_lag3 = last_obs$GS10TB3Mx_lag2[3],
    UMCSENTx_lag2 = last_obs$UMCSENTx_lag1[3],
    UMCSENTx_lag3 = last_obs$UMCSENTx_lag2[3],
    UMCSENTx_lag4 = last_obs$UMCSENTx_lag3[3],
    UMCSENTx_lag5 = last_obs$UMCSENTx_lag4[3]
  )
head
q1_ardl <- transformed_ardl[-c(1:9),] %>%
  model(TSLM(GDPC1 ~ GDPC1_lag2 + CLAIMSx_lag2 + CLAIMSx_lag3 + CLAIMSx_lag4 + CLAIMSx_lag5 + ANDENOx_lag2 + ANDENOx_lag3 + GS10TB3Mx_lag2 + GS10TB3Mx_lag3 + UMCSENTx_lag2 + UMCSENTx_lag3 + UMCSENTx_lag4 + UMCSENTx_lag5))
ardl_fc_q1 <- q1_ardl %>%
  fabletools::forecast(future_data1)
hilo(q1_fc_ardl, 50) %>%
  dplyr::select(.mean, '50%')

### VAR
var_fc <- transformed %>%
  model(var1 = VAR(vars(GDPC1, CLAIMSx, UMCSENTx, GS10TB3Mx, ANDENOx) ~ AR(1))) %>%
  fabletools::forecast()

hilo(var_fc, 50) %>%
  dplyr::select(.mean, '50%')

mean_forecast_q1 <- (ar1_fc[2,4] + var_final[[4]][2] + ardl_fc_q1[4])/3
print(sprintf("The mean forecast growth for Q1 2025 is %s%%", round(mean_forecast_q1,2)))

.model,date,GDPC1,.mean
<chr>,<qtr>,<dist>,<dbl>
ar1,2024 Q4,"N(2.7, 19)",2.687118
ar1,2025 Q1,"N(2.7, 19)",2.687968
ar1,2025 Q2,"N(2.7, 19)",2.687966
ar1,2025 Q3,"N(2.7, 19)",2.687966
ar1,2025 Q4,"N(2.7, 19)",2.687966


function (x, ...) 
UseMethod("head")
<bytecode: 0x56ab2f654308>
<environment: namespace:utils>

.mean,50%,date
<dbl>,<hilo>,<qtr>
0.4951366,"[-2.392346, 3.382619]50",2025 Q1


  .mean    <NA>     <NA>      <NA>       <NA>       50%$GDPC1              
1 1.082540 6.783605 -3.082020 -0.9723130 -3.7650265 [-1.608111, 3.773192]50
2 1.181692 6.401041 -1.744071 -0.6850986 -1.3195331 [-1.734628, 4.098011]50
3 1.352452 5.950651 -1.870099 -0.4568163 -1.6794934 [-1.582727, 4.287631]50
4 1.447969 5.436091 -1.629722 -0.2453853 -1.3565503 [-1.493977, 4.389915]50
5 1.583093 4.868580 -1.488598 -0.0582414 -1.2397654 [-1.364342, 4.530529]50
6 1.696534 4.373873 -1.338555  0.1092989 -1.0731694 [-1.255233, 4.648300]50
7 1.799930 3.929687 -1.212381  0.2585118 -0.9419102 [-1.155286, 4.755145]50
8 1.891384 3.534700 -1.097695  0.3916158 -0.8198984 [-1.066568, 4.849337]50
  50%$CLAIMSx             50%$UMCSENTx             50%$GS10TB3Mx             
1 [-3.036191, 16.60340]50 [-6.584493, 0.4204531]50 [-1.3494992, -0.5951267]50
2 [-4.453254, 17.25534]50 [-5.365405, 1.8772625]50 [-1.2098170, -0.1603802]50
3 [-4.969712, 16.87101]50 [-5.506413, 1.7662154]50 [-1.0750515,  0.1614188]50
4 [-

[1] "The mean forecast growth for Q1 2025 is 1.45%"


In [49]:
## Q2 2025
### ARDL using direct h-step forecasting
transformed_ardl <- transformed %>%
  mutate(across(c(GDPC1, GS10TB3Mx, CLAIMSx, ANDENOx, UMCSENTx),
                list(lag5 = ~ lag(., 5), lag6 = ~ lag(., 6),lag7 = ~ lag(., 7), lag8 = ~ lag(., 8), lag9 = ~ lag(., 9)),
                .names = "{col}_{fn}"))


future_data2 <- new_data(transformed, n = 1, keep_all=TRUE) %>%
  mutate(
    date = yearquarter("2025 Q2"),
    GDPC1_lag3 = last_obs$GDPC1[3],
    CLAIMSx_lag3 = last_obs$CLAIMSx_lag1[3],
    CLAIMSx_lag4 = last_obs$CLAIMSx_lag2[3],  # Correctly getting the t-2 value
    CLAIMSx_lag5 = last_obs$CLAIMSx_lag3[3],
    CLAIMSx_lag6 = last_obs$CLAIMSx_lag4[3],
    ANDENOx_lag3 = last_obs$ANDENOx_lag1[3],
    ANDENOx_lag4 = last_obs$ANDENOx_lag2[3],
    GS10TB3Mx_lag3 = last_obs$GS10TB3Mx_lag1[3],
    GS10TB3Mx_lag4 = last_obs$GS10TB3Mx_lag2[3],
    UMCSENTx_lag3 = last_obs$UMCSENTx_lag1[3],
    UMCSENTx_lag4 = last_obs$UMCSENTx_lag2[3],
    UMCSENTx_lag5 = last_obs$UMCSENTx_lag3[3],
    UMCSENTx_lag6 = last_obs$UMCSENTx_lag4[3]
  )
head
q2_ardl <- transformed_ardl[-c(1:9),] %>%
  model(TSLM(GDPC1 ~ GDPC1_lag3 + CLAIMSx_lag3 + CLAIMSx_lag4 + CLAIMSx_lag5 + CLAIMSx_lag6 + ANDENOx_lag3 + ANDENOx_lag4 + GS10TB3Mx_lag3 + GS10TB3Mx_lag4 + UMCSENTx_lag3 + UMCSENTx_lag4 + UMCSENTx_lag5 + UMCSENTx_lag6))
ardl_fc_q2 <- q2_ardl %>%
  fabletools::forecast(future_data2)
hilo(ardl_fc_q2, 50) %>%
  dplyr::select(.mean, '50%')

mean_forecast_q2 <- (ar1_fc[3,4] + var_final[[4]][3] + ardl_fc_q2[4])/3
print(sprintf("The mean forecast growth for Q2 2025 is %s%%", round(mean_forecast_q2,2)))

### Since our VAR and AR forecasts are already iterated through Q4 2025, we do not need to do a separate forecast and will just take the corresponding values from the forecast earlier.

function (x, ...) 
UseMethod("head")
<bytecode: 0x56ab2f654308>
<environment: namespace:utils>

.mean,50%,date
<dbl>,<hilo>,<qtr>
-0.006035769,"[-3.092566, 3.080495]50",2025 Q2


[1] "The mean forecast growth for Q2 2025 is 1.34%"


In [48]:
## Q4 2025
### ARDL using direct h-step forecasting
transformed_ardl <- transformed %>%
  mutate(across(c(GDPC1, GS10TB3Mx, CLAIMSx, ANDENOx, UMCSENTx),
                list(lag5 = ~ lag(., 5), lag6 = ~ lag(., 6),lag7 = ~ lag(., 7), lag8 = ~ lag(., 8), lag9 = ~ lag(., 9)),
                .names = "{col}_{fn}"))


future_data2 <- new_data(transformed, n = 1, keep_all=TRUE) %>%
  mutate(
    date = yearquarter("2025 Q2"),
    GDPC1_lag5 = last_obs$GDPC1[3],
    CLAIMSx_lag5 = last_obs$CLAIMSx_lag1[3],
    CLAIMSx_lag6 = last_obs$CLAIMSx_lag2[3],  # Correctly getting the t-2 value
    CLAIMSx_lag7 = last_obs$CLAIMSx_lag3[3],
    CLAIMSx_lag8 = last_obs$CLAIMSx_lag4[3],
    ANDENOx_lag5 = last_obs$ANDENOx_lag1[3],
    ANDENOx_lag6 = last_obs$ANDENOx_lag2[3],
    GS10TB3Mx_lag5 = last_obs$GS10TB3Mx_lag1[3],
    GS10TB3Mx_lag6 = last_obs$GS10TB3Mx_lag2[3],
    UMCSENTx_lag5 = last_obs$UMCSENTx_lag1[3],
    UMCSENTx_lag6 = last_obs$UMCSENTx_lag2[3],
    UMCSENTx_lag7 = last_obs$UMCSENTx_lag3[3],
    UMCSENTx_lag8 = last_obs$UMCSENTx_lag4[3]
  )
head
q4_ardl <- transformed_ardl[-c(1:9),] %>%
  model(TSLM(GDPC1 ~ GDPC1_lag5 + CLAIMSx_lag5 + CLAIMSx_lag6 + CLAIMSx_lag7 + CLAIMSx_lag8 + ANDENOx_lag5 + ANDENOx_lag6 + GS10TB3Mx_lag5 + GS10TB3Mx_lag6 + UMCSENTx_lag5 + UMCSENTx_lag6 + UMCSENTx_lag7 + UMCSENTx_lag8))
ardl_fc_q4 <- q4_ardl %>%
  fabletools::forecast(future_data2)
hilo(ardl_fc_q4, 50) %>%
  dplyr::select(.mean, '50%')

mean_forecast_q4 <- (ar1_fc[5,4] + var_final[[4]][5] + ardl_fc_q4[4])/3
print(sprintf("The mean forecast growth for Q4 2025 is %s%%", round(mean_forecast_q4,2)))

### Since our VAR and AR forecasts are already iterated through Q4 2025, we do not need to do a separate forecast and will just take the corresponding values from the forecast earlier.

function (x, ...) 
UseMethod("head")
<bytecode: 0x56ab2f654308>
<environment: namespace:utils>

.mean,50%,date
<dbl>,<hilo>,<qtr>
1.168397,"[-1.938538, 4.275333]50",2025 Q2


[1] "The mean forecast growth for Q4 2025 is 1.81 %"
